In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import statsmodels.api as sm

In [2]:
krx = '/data/hun/KRX_marketdata/'

## 1. 파일 가져오기

#### Ticker

In [3]:
#이건 industry code가 없음
tickers = pd.read_csv(krx+'ticker_recent.csv', encoding ='cp949')
tickers.columns = ('full_ticker', 'ticker', 'name_full', 'name',
              'name_eng', 'listed_data', 'market', 'security_category',
              'related_department', 'preferred', 'face_value', 'shares')
tickers = tickers.drop(['related_department'], axis=1).dropna().reset_index(drop=True)

#### Price

In [23]:
prices = pd.read_csv(krx+"krx_marketdata_monthly.csv")
prices['ticker'] = prices.ticker.apply(lambda x: '0'*(6-len(str(x)))+str(x))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
prices = pd.merge(prices, tickers[['ticker','shares']], how = 'left', on='ticker')
prices.dropna(inplace = True)

#### Finance

In [31]:
fin = pd.read_csv(krx+"krx_fin_monthly2.csv")
fin['date'] = fin.date.apply(lambda x: f'{str(x)[:4]}/{str(x)[5:7]}')
fin['ticker'] = fin.ticker.apply(lambda x: '0'*(6-len(str(x)))+str(x))
fin.drop(['close','daily_change','daily_return','date2'], axis=1, inplace = True)
fin.rename(columns={'주당배당금': 'div', '배당수익률':'div_ret'}, inplace = True)

In [32]:
prices = pd.merge(prices, fin[["date","ticker","PER","PBR","div_ret"]], how='left', on=['ticker','date'])

#### Macro

In [35]:
#이자율
interest1 = pd.read_excel(krx+'stat_107302.xls')
interest2 = pd.read_excel(krx+'stat_107301.xls')
interest = pd.concat([interest1.transpose(),interest2.transpose()]).reset_index()
int_col = ['date','tb3y','tb5y','tb10y','cb3y','cd3m','call','br']
interest.columns = int_col
interest.drop([0],axis=0, inplace=True)
interest["date"] = interest.date.apply(lambda x: f'{x[:4]}/{x[4:6]}')
interest.replace('-', np.nan, inplace=True)

In [36]:
interest.drop(['tb10y','br'], axis=1, inplace =True)

In [38]:
prices = pd.merge(prices, interest, how = 'left', on ='date')

In [80]:
prices[interest.columns[1:]] = prices[interest.columns[1:]].astype('float')

In [39]:
# bpr = nan인 경우, 우량주 등
prices.drop(prices[prices.PBR.isna()].index, inplace = True)

## 팩터 전처리

In [475]:
# prices.to_csv(krx+'price_before_factor_from2010', index = False)

In [41]:
prices['size'] = prices.close * prices.shares

In [42]:
prices['ret'] = 0
for i in tqdm(tickers.ticker):
    stock = prices[prices.ticker == i]
    ind = stock.index
    prices.loc[ind, 'ret'] = stock.close.pct_change(-1).shift(1)

100%|██████████| 2600/2600 [01:00<00:00, 43.32it/s]


In [43]:
dates = prices.date.unique()
len(dates)

297

In [44]:
def size_bin(data: pd.Series) -> pd.Series:
    size_rnk = data.rank(pct=True)
    size_bin = pd.cut(size_rnk, bins=[0, 0.2, 0.4, 0.6, 0.8, 1], include_lowest=True, labels=[1,2,3,4,5])
    return size_bin.astype('int')

def size_bin_factor(data: pd.Series) -> pd.Series:
    size_rnk = data.rank(pct=True)
    size_bin = pd.cut(size_rnk, bins=[0, 0.5, 1], include_lowest=True, labels=[1,2])
    return size_bin.astype('int')

In [45]:
##시총가중 월별수익률
def size_weighted_return(data: pd.DataFrame) -> int:
    size_sum = data['size'].sum()
    weighted_return = data.apply(lambda x: x['size']*x['ret'], axis=1).sum()
    return weighted_return/size_sum

In [52]:
# size_bin, size_weighted_return
prices['size_bin'] = 0
prices['SW_ret'] = 0
for i in tqdm(dates):
    stock = prices[prices.date == i]
    if len(stock) ==0:
        continue
    else:
        ind = stock.index
        prices.loc[ind, 'size_bin'] =  size_bin(stock['size'])
        prices.loc[ind, 'SW_ret'] = size_weighted_return(stock)

100%|██████████| 297/297 [00:14<00:00, 20.77it/s]


In [66]:
prices['RM_RF'] = prices['SW_ret'] - prices['cd3m'].astype('float')/1200

In [67]:
def inv(x):
    if x != 0:
        return 1/x
    else:
        return 0
def HML_bin(data: pd.Series) -> pd.Series:
    data = data.apply(inv)
    size_rnk = data.rank(pct=True)
    size_bin = pd.cut(size_rnk, bins=[0, 0.2, 0.4, 0.6, 0.8, 1], include_lowest=True, labels=[1,2,3,4,5])
    try:
        return size_bin.astype('int')
    except:
        print(data[data == np.NaN])
def HML_bin_factor(data: pd.Series) -> pd.Series:
    data = data.apply(inv)
    size_rnk = data.rank(pct=True)
    size_bin = pd.cut(size_rnk, bins=[0, 0.4, 0.7, 1], include_lowest=True, labels=[1,2,3])
    return size_bin.astype('int')

In [68]:
# HML bin
prices.bpr_bin = 0
prices.bpr_bin_factor = 0
prices.size_bin_factor = 0
for i in tqdm(dates):
    stock = prices[prices.date == i]
    ind = stock.index
    prices.loc[ind, 'bpr_bin'] =  HML_bin(stock['PBR'])
    prices.loc[ind, 'bpr_bin_factor'] =  HML_bin_factor(stock['PBR'])
    prices.loc[ind, 'size_bin_factor'] =  size_bin_factor(stock['size'])

100%|██████████| 297/297 [00:10<00:00, 28.47it/s]


In [69]:
prices['smb*hml'] = prices.apply(lambda x: (x.size_bin_factor-1)*3 + x.bpr_bin_factor, axis=1)

Beta

In [88]:
def get_beta(data: pd.DataFrame):
    X = data.RM_RF.astype('float')
    Y = data.ret
    X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    params = results.params
    if len(params==1):
        params['const']=0
    mse_resid = results.mse_resid
    return params, mse_resid

In [89]:
prices['beta'] = 0
prices['const'] = 0
prices['ido_vol'] = 0
for i in tqdm(tickers.ticker):
    stock = prices[prices.ticker == i]
    if len(stock) <= 1:
        if len(stock) ==1 :
            print(stock.ticker[0], stock.name[0])
            pass
    else:
        ind = stock.index[1:]
        stock = stock.loc[ind]
        for j in range(len(ind),0,-1):
            ind_temp = ind[max(j-12,0):j]
            temp = stock.loc[ind_temp]
            params, mse_resid = get_beta(temp)
            prices.loc[ind_temp, 'const'] = params['const']
            prices.loc[ind_temp, 'beta'] = params['RM_RF']
            prices.loc[ind_temp, 'ido_vol'] = mse_resid
prices['ido_vol'] = prices['ido_vol'].fillna(0)

100%|██████████| 2600/2600 [51:04<00:00,  1.18s/it] 


In [90]:
prices.to_csv(krx+'prices_final.csv',index=False)

## 팩터 포트폴리오

In [279]:
port = pd.DataFrame(columns = [1,2,3,4,5], index = [1,2,3,4,5])
port.columns.name = 'BPR'
port.index.name = 'Size'

In [280]:
port

BPR,1,2,3,4,5
Size,,,,,
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN


## 팩터 계산

In [70]:
factors = prices[['date','ticker','name','ret']].copy()

In [71]:
# factors.to_csv(krx+'factors+until2010.csv')

In [72]:
factors

,date,ticker,name,ret
0,2021/09,098120,마이크로컨텍솔,NaN
1,2021/08,098120,마이크로컨텍솔,-0.114160
2,2021/07,098120,마이크로컨텍솔,0.130273
3,2021/06,098120,마이크로컨텍솔,0.034660
4,2021/05,098120,마이크로컨텍솔,0.209627
...,...,...,...,...
430267,2017/11,238490,힘스,-0.106343
430268,2017/10,238490,힘스,0.237875
430269,2017/09,238490,힘스,0.021226
430270,2017/08,238490,힘스,-0.179884


In [81]:
#RM-RF
factors['RM_RF'] = prices['RM_RF']

In [103]:
def smb_hml(data):
    data = data.dropna(subset = ['ret'])
    res = []
    for i in range(1,7):
        res.append(data[data['smb*hml'] == i].ret.mean())
    smb = np.mean(res[:3]) - np.mean(res[3:])
    hml = (res[2]+res[5])/2 - (res[0]+res[3])/2
    return res, smb, hml

In [110]:
### HML, SMB
prices.smb = 0
factors.hml = 0
smb_hml_port = pd.DataFrame(columns = ["SL", "SM", "SH", "BL", "BM", "BH"])
for i in tqdm(dates):
    stock = prices[prices.date == i]
    ind = stock.index
    res, smb, hml = smb_hml(stock)
    factors.loc[ind, 'smb'] = smb
    factors.loc[ind, 'hml'] = hml
# factors.to_csv(krx+"ff3.csv", index = False)

  2%|▏         | 6/297 [00:00<00:10, 27.73it/s]

2021/09 nan nan
2021/08 -0.005351680756814747 0.03771549528274665
2021/07 -0.023582022247856948 -0.0042843020641569265
2021/06 -0.0008562602206512084 -0.020802207125245248
2021/05 0.03937651286454319 0.020717338638420103
2021/04 0.11671746496550137 0.06037768775197677
2021/03 0.05208198489480465 0.032682897484419965


  4%|▍         | 12/297 [00:00<00:09, 28.61it/s]

2021/02 0.0496615469684615 0.07460991985843698
2021/01 0.004102893299664501 0.05050261742661352
2020/12 0.035239475986280605 0.05512396470125142
2020/11 -0.017059823328327783 0.019468297427194813
2020/10 -0.013116964687386393 0.0256596496738129
2020/09 0.05742883746115121 0.05091469930619171
2020/08 0.04675258906823273 0.00808644783254273


  6%|▌         | 18/297 [00:00<00:09, 29.14it/s]

...

1998/07 nan nan
1998/06 nan nan
1998/04 nan nan
1998/03 nan nan
1998/02 nan nan
1998/01 nan nan
1997/12 nan nan
1997/11 nan nan


 99%|█████████▉| 294/297 [00:09<00:00, 32.28it/s]

1998/05 nan nan
1997/09 nan nan
1997/07 nan nan
1997/06 nan nan
1997/05 nan nan
1997/04 nan nan
1997/03 nan nan
1997/02 nan nan


100%|██████████| 297/297 [00:09<00:00, 30.87it/s]

1997/01 nan nan
1997/10 nan nan
1997/08 nan nan


,date,ticker,name,close,high,low,total_volume,avg_volume,total_transactions,avg_transactions,...,size_bin,SW_ret,RM_RF,bpr_bin,bpr_bin_factor,size_bin_factor,smb*hml,beta,const,ido_vol


In [119]:
# factors.to_csv(krx+"ff3.csv", index = False)

In [86]:
# prices.to_csv(krx+'ff3_prices.csv', index=False)

In [91]:
len(factors), len(prices)

(418562, 418562)

In [99]:
factors['div_ret'] = factors.div_ret.fillna(0)

In [93]:
# Dividend to price
factors['div_ret'] = prices['div_ret']

In [139]:
#Beta, Beta_squared
factors['beta'] = prices['beta']
factors['beta_seq'] = prices['beta']**2

In [541]:
len(factors), len(prices)

(205304, 205304)

In [140]:
#Idiosyncratic return volatiliy, FF3 return vol
factors['ido_vol'] = prices['ido_vol']

In [120]:
#EPR (PER 역수)
factors['EPR'] = prices['PER'].apply(inv)
factors['BPR'] = prices['PBR'].apply(inv)

In [131]:
#Size
prices['size_factor'] =0
for i in tqdm(dates):
    stock = prices[prices.date == i]
    ind = stock.index
    size_rnk = stock["size"].rank(pct=True)
    prices.loc[ind, 'size_factor'] = size_rnk

100%|██████████| 297/297 [00:07<00:00, 41.22it/s]


In [134]:
factors['size']=prices["size_factor"]

In [136]:
#Share turnover = transaction/supply
factors['share_turn'] =0
for i in tqdm(tickers.ticker):
    share = tickers[tickers.ticker ==i].shares.values[0]
    stock = prices[prices.ticker == i]
    ind = stock.index
    factors.loc[ind, 'share_turn'] = stock.total_volume/share

100%|██████████| 2600/2600 [01:00<00:00, 43.04it/s]


In [137]:
#Momentum 1m, 12m, 36m, 6m, change in 6m momentum
factors['mom1'] =0
factors['mom6'] =0
factors['mom12'] =0
for i in tqdm(tickers.ticker):
    stock = prices[prices.ticker == i]
    ind = stock.index
    temp = stock[::-1]
    mom1 = stock.ret.shift(-1)
    mom6 = temp.ret.rolling(6, min_periods=1).mean()[::-1]
    mom12 = temp.ret.rolling(12, min_periods=1).mean()[::-1]
    factors.loc[ind, 'mom1'] = mom1
    factors.loc[ind, 'mom6'] = mom6
    factors.loc[ind, 'mom12'] = mom12

100%|██████████| 2600/2600 [01:10<00:00, 36.96it/s]


In [142]:
factors.to_csv(krx+'factors.csv', index=False)